In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,866 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,670 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3,346 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,369 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/pp

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show the delayed flight data.
home_sales_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales')

In [8]:
# Importing more dependencies
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [37]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT
  YEAR(date) AS year,
  ROUND(AVG(price) ,2) AS average_price
FROM home_sales
WHERE bedrooms=4
GROUP BY 1
ORDER BY 1 ASC
""").show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [39]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("""
SELECT
  YEAR(date_built) AS YEAR_BUILT,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY 1
ORDER BY YEAR_BUILT ASC
""").show()

+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [40]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""
SELECT
  YEAR(date_built) AS YEAR_BUILT,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
AND bathrooms = 3
AND floors = 2
AND sqft_living >= 2000
GROUP BY 1
ORDER BY YEAR_BUILT ASC
""").show()


+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [54]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
SELECT
  view AS VIEW_RATE,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY 1
HAVING AVERAGE_PRICE >= 350000
ORDER BY view desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+-------------+
|VIEW_RATE|AVERAGE_PRICE|
+---------+-------------+
|       99|   1061201.42|
|       98|   1053739.33|
|       97|   1129040.15|
|       96|   1017815.92|
|       95|    1054325.6|
|       94|    1033536.2|
|       93|   1026006.06|
|       92|    970402.55|
|       91|   1137372.73|
|       90|   1062654.16|
|       89|   1107839.15|
|       88|   1031719.35|
|       87|    1072285.2|
|       86|   1070444.25|
|       85|   1056336.74|
|       84|   1117233.13|
|       83|   1033965.93|
|       82|    1063498.0|
|       81|   1053472.79|
|       80|    991767.38|
+---------+-------------+
only showing top 20 rows

--- 0.6328043937683105 seconds ---


In [55]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [56]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [57]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT
  view AS VIEW_RATE,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY 1
HAVING AVERAGE_PRICE >= 350000
ORDER BY view desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------+-------------+
|VIEW_RATE|AVERAGE_PRICE|
+---------+-------------+
|       99|   1061201.42|
|       98|   1053739.33|
|       97|   1129040.15|
|       96|   1017815.92|
|       95|    1054325.6|
|       94|    1033536.2|
|       93|   1026006.06|
|       92|    970402.55|
|       91|   1137372.73|
|       90|   1062654.16|
|       89|   1107839.15|
|       88|   1031719.35|
|       87|    1072285.2|
|       86|   1070444.25|
|       85|   1056336.74|
|       84|   1117233.13|
|       83|   1033965.93|
|       82|    1063498.0|
|       81|   1053472.79|
|       80|    991767.38|
+---------+-------------+
only showing top 20 rows

--- 0.35890841484069824 seconds ---


In [58]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [59]:
# 11. Read the parquet formatted data.
df_partition = spark.read.parquet('home_sales_partitioned')

In [60]:
# 12. Create a temporary table for the parquet data.
df_partition.createOrReplaceTempView('p_home_sales')

In [61]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""
SELECT
  view AS VIEW_RATE,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM p_home_sales
GROUP BY 1
HAVING AVERAGE_PRICE >= 350000
ORDER BY view desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------+-------------+
|VIEW_RATE|AVERAGE_PRICE|
+---------+-------------+
|       99|   1061201.42|
|       98|   1053739.33|
|       97|   1129040.15|
|       96|   1017815.92|
|       95|    1054325.6|
|       94|    1033536.2|
|       93|   1026006.06|
|       92|    970402.55|
|       91|   1137372.73|
|       90|   1062654.16|
|       89|   1107839.15|
|       88|   1031719.35|
|       87|    1072285.2|
|       86|   1070444.25|
|       85|   1056336.74|
|       84|   1117233.13|
|       83|   1033965.93|
|       82|    1063498.0|
|       81|   1053472.79|
|       80|    991767.38|
+---------+-------------+
only showing top 20 rows

--- 0.6149544715881348 seconds ---


In [62]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [63]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False